### Load Data

In [1]:
# import numpy & pandas
import numpy as np
import pandas as pd
import psycopg2 
import time
import matplotlib.pyplot as plt 
import statsmodels.formula.api as smf
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
import rpy2.ipython
%load_ext rpy2.ipython

In [3]:
%%R
library(ggplot2)
library("MatchIt") # available on conda-forge!
library("cobalt")
library("quickmatch")
library("marginaleffects") # available on conda-forge!
library("sensemakr") # install.packages('sensemakr','/Users/kikwon/miniconda3/envs/statsenv1/lib/R/library')
library('tipr')

 cobalt (Version 4.5.1, Build Date: 2023-04-27)

Attaching package: ‘cobalt’

The following object is masked from ‘package:MatchIt’:

    lalonde

Loading required package: distances
Please cite the `quickmatch` package as:
   Savje, Fredrik, Michael J. Higgins and Jasjeet S. Sekhon (2017),
   "Generalized Full Matching", arXiv 1703.03882.
   https://arxiv.org/abs/1703.03882
See details in:
Carlos Cinelli and Chad Hazlett (2020). Making Sense of Sensitivity: Extending Omitted Variable Bias. Journal of the Royal Statistical Society, Series B (Statistical Methodology).


In [4]:
conn=psycopg2.connect(dbname= 'ancestry', host='bit-dw.cljpbhsretnx.us-east-1.redshift.amazonaws.com', port= '5439', 
                      user= 'zwu', password= 'Ancestry1')
sql = """
select 
*
from a.zwu_impact_sc_colla_invite_retention
where usertype = 'Subs'
    """

df = pd.read_sql(sql, con = conn)

In [5]:
df.head()

,ucdmid,tvalueid,start_date,end_date,previous_month_start_date,usertype,retain,year_old,dna_segment,countrysubregiondescription,...,nodes_creation_flag,num_trees_creation,trees_creation_flag,hints_accepted,hints_accepted_flag,num_active_days,active_days_flg,num_page_view,page_view_flg,sc_colla_invite_prev_flg
0,00785C1F-0002-0000-0000-000000000000,5,2023-01-30,2023-03-01,2022-12-31,Subs,1,46+,DNA,US,...,0,0,0,6,1,0,0,0,0,0
1,00785C1F-0002-0000-0000-000000000000,6,2023-08-02,2023-09-01,2023-07-03,Subs,1,46+,DNA,US,...,0,0,0,6,1,0,0,0,0,0
2,00CD4C98-0002-0000-0000-000000000000,4,2023-07-17,2023-08-16,2023-06-17,Subs,1,46+,Non-DNA,US,...,1,0,0,0,0,0,0,0,0,0
3,00CD4C98-0002-0000-0000-000000000000,3,2023-01-17,2023-02-16,2022-12-18,Subs,1,46+,Non-DNA,US,...,1,0,0,0,0,0,0,0,0,0
4,01103ADC-0001-0000-0000-000000000000,3,2023-04-04,2023-05-03,2023-03-05,Subs,1,36-45,Non-DNA,US,...,1,0,0,108,1,4,1,5,1,0


### A note on dealing with missing data in causal inference

In [6]:
df.columns[df.isnull().any()]

Index([], dtype='object')

In [7]:
# # remove rows with null values
# df = df[df.emptynull==0]

### Final reprocessing

### Naive/Simple differences in means / risk ratio

In [8]:
df.groupby(['sc_colla_invite_flg'])['retain'].agg(['mean','count','sum'])

,mean,count,sum
sc_colla_invite_flg,,,
0,0.85299,15188799,12955896
1,0.86296,1332859,1150204


In [9]:
%%R -i df
m.out1 <- matchit(sc_colla_invite_flg ~ year_old + dna_segment + countrysubregiondescription + subscriptiongroup + usage_device + users_life_type + num_nodes_creation + num_trees_creation + hints_accepted + num_active_days + num_page_view + sc_colla_invite_prev_flg, 
                  data = df,
                  method ='cem',
                  estimand = 'ATE',
                  cutpoints = list(num_nodes_creation=c(0,1,3,7,15,31,63,100,500,1000), # 1st gen, 2nd gen, 3rd gen, 4th gen, 5th gen, 6th gen...
                                   num_trees_creation = c(0,1,5,10,50,100,500,1000),
                                   hints_accepted = c(0,10,20,30,40,50,100,500,1000),
                                   num_active_days = c(0,1,2,3,4,5,10,15,20,25,30),
                                   num_page_view = c(0,10,20,30,40,50,100,500,1000)
                                  )
#                   grouping = list(usertypegroupdescription=list(c("Current Subscriber","Current Free Subscriber")))
                 )
m.out1

A matchit object
 - method: Coarsened exact matching
 - number of obs.: 16521658 (original), 12491095 (matched)
 - target estimand: ATE
 - covariates: year_old, dna_segment, countrysubregiondescription, subscriptiongroup, usage_device, users_life_type, num_nodes_creation, num_trees_creation, hints_accepted, num_active_days, num_page_view, sc_colla_invite_prev_flg


In [11]:
# %%R
# love.plot(m.out1, 
#           binary='std',
#           drop.distance = TRUE,
#           ##abs=TRUE,
#           var.order = 'unadjusted',
# #           addl = ~ I(num_nodes==0)+I(num_nodes==1)+I(log(num_nodes+1))+I(num_nodes^2)+I(num_objects==0)+I(num_objects==1)+I(log(num_objects+1))+I(num_objects^2),
#           thresholds=c(m=.1))

In [12]:
# %%R
# # density plot for discrete variable Gender
# bal.plot(m.out1, var.name = 'gender', which = 'both')

In [13]:
# %%R
# #eQQ plot
# plot(m.out1, type = "qq", which.xs = ~num_nodes_creation + num_trees_creation + hints_accepted + num_active_days + num_page_view)

In [14]:
%%R -o md
md <- match.data(m.out1)

In [15]:
%%R
fit <- glm(retain ~ sc_colla_invite_flg * (year_old + dna_segment + countrysubregiondescription + subscriptiongroup + usage_device + users_life_type + num_nodes_creation + num_trees_creation + hints_accepted + num_active_days + num_page_view + sc_colla_invite_prev_flg), 
           data = md, 
           weights = weights,
           family=quasibinomial())

avg_comparisons(fit,
                variables = "sc_colla_invite_flg",
                vcov = "HC3",
                wts = "weights",
                comparison='lnratioavg',
                transform='exp')

Error: Unable to compute predicted values with this model. This error can arise
  when `insight::get_data()` is unable to extract the dataset from the
  model object, or when the data frame was modified since fitting the
  model. You can try to supply a different dataset to the `newdata`
  argument.
  
  Bug Tracker:
  https://github.com/vincentarelbundock/marginaleffects/issues
In addition: Warning message:
Unable to extract a variance-covariance matrix using this `vcov`
  argument. Standard errors are computed using the default variance
  instead. Perhaps the model or argument is not supported by the
  `sandwich` or `clubSandwich` packages. If you believe that the model is
  supported by one of these two packages, you can open a feature request
  on Github. 
Error: no more error handlers available (recursive errors?); invoking 'abort' restart
Error: Unable to compute predicted values with this model. This error can arise
  when `insight::get_data()` is unable to extract the dataset f

RInterpreterError: Failed to parse and evaluate line 'fit <- glm(retain ~ sc_colla_invite_flg * (year_old + dna_segment + countrysubregiondescription + subscriptiongroup + usage_device + users_life_type + num_nodes_creation + num_trees_creation + hints_accepted + num_active_days + num_page_view + sc_colla_invite_prev_flg), \n           data = md, \n           weights = weights,\n           family=quasibinomial())\n\navg_comparisons(fit,\n                variables = "sc_colla_invite_flg",\n                vcov = "HC3",\n                wts = "weights",\n                comparison=\'lnratioavg\',\n                transform=\'exp\')\n'.
R error message: 'Error: Unable to compute predicted values with this model. This error can arise\n  when `insight::get_data()` is unable to extract the dataset from the\n  model object, or when the data frame was modified since fitting the\n  model. You can try to supply a different dataset to the `newdata`\n  argument.\n  \n  Bug Tracker:\n  https://gith

In [ ]:
%%R
summary(lm(retain ~ sc_colla_invite_flg, data = md, weights = weights))$coef